In [1]:
!cp -r ../input/recsys-repo/RecSys_Course_AT_PoliMi-master/* ./
%config Completer.use_jedi = False

import pandas as pd
import numpy as np
from scipy import sparse as sps
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.BaseRecommender import BaseRecommender

In [2]:
def get_URM():
    return pd.read_csv('/kaggle/input/urm-true-binary/URM_True_Binary.csv')
URM_all_dataframe = get_URM()

In [3]:
URM_path = "../input/urm-true-binary/URM_True_Binary.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path, 
                                sep=",",
                                dtype={0:int, 1:int, 2:float},
                                header=0)
URM_all_dataframe.columns = ["UserID", "ItemID", "Data"]
URM_all = sps.coo_matrix((URM_all_dataframe["Data"].values, 
                          (URM_all_dataframe["UserID"].values, URM_all_dataframe["ItemID"].values)))
URM_all = URM_all.tocsr() # to obtain fast access to rows (users)

In [4]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

# split data into train and validation data 80/20
URM_train, URM_valid = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [5]:
rec = EASE_R_Recommender(URM_train)

In [6]:
rec.fit()

EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 10.59 min


In [7]:
evaluator_validation = EvaluatorHoldout(URM_valid, cutoff_list=[10])

EvaluatorHoldout: Ignoring 298 ( 0.7%) Users that have less than 1 test interactions


In [8]:
result_df, _ = evaluator_validation.evaluateRecommender(rec)
result_df

EvaluatorHoldout: Processed 41331 (100.0%) in 55.55 sec. Users per second: 744


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.064475,0.088531,0.07705,0.032111,0.04351,0.201352,0.091128,0.070204,0.398781,0.251765,...,0.992842,0.395926,0.992842,0.008679,7.773179,0.988591,0.017092,0.576091,5.311766,0.244134


In [9]:
test_users = pd.read_csv('/kaggle/input/competition-data/data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
41111,41624
41112,41625
41113,41626
41114,41627


In [10]:
user_id = test_users['user_id']
recommendations = []
for user in user_id:
    recommendations.append(rec.recommend(user,cutoff = 10,remove_seen_flag = True))
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])
    
test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])
test_users.to_csv('submissionR_NoDup.csv', index=False)

In [11]:
ICM_type_df = pd.read_csv("/kaggle/input/competition-data/data_ICM_type.csv")
ICM_type_df

,item_id,feature_id,data
0,0,1,1
1,1,3,1
2,2,4,1
3,3,1,1
4,4,3,1
...,...,...,...
23086,27963,1,1
23087,27964,2,1
23088,27965,1,1
23089,27966,1,1


In [12]:
items = ICM_type_df.item_id
features = ICM_type_df.feature_id
data = ICM_type_df.data
ICM_type = sps.csr_matrix((data, (items, features)))
ICM_type = ICM_type.astype(dtype = np.int32)
ICM_type.shape

(27968, 8)

In [13]:
n_users = 41629
n_itemsFromICM = ICM_type.shape[0]
URM_path = "../input/urm-true-binary/URM_True_Binary.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path, 
                                sep=",",
                                dtype={0:int, 1:int, 2:float},
                                header=0)
URM_all_dataframe.columns = ["UserID", "ItemID", "Data"]
URM_all = sps.coo_matrix((URM_all_dataframe["Data"].values, 
                          (URM_all_dataframe["UserID"].values, URM_all_dataframe["ItemID"].values)), shape=(n_users,n_itemsFromICM))
URM_all = URM_all.tocsr() # to obtain fast access to rows (users)

In [14]:
stacked_URM_type = sps.vstack([URM_all, ICM_type.T])
stacked_URM_type = sps.csr_matrix(stacked_URM_type)
stacked_URM_type

<41637x27968 sparse matrix of type '<class 'numpy.float64'>'
	with 1577731 stored elements in Compressed Sparse Row format>

In [15]:
ICM_length_df = pd.read_csv("/kaggle/input/competition-data/data_ICM_length.csv")
ICM_length_df

,item_id,feature_id,data
0,0,0,1
1,1,0,1
2,2,0,21
3,3,0,1
4,4,0,1
...,...,...,...
23086,27963,0,1
23087,27964,0,1
23088,27965,0,1
23089,27966,0,1


In [16]:
items = ICM_length_df.item_id
features = ICM_length_df.feature_id
data = ICM_length_df.data
ICM_length = sps.csr_matrix((data, (items, features)))
ICM_length = ICM_length.astype(dtype = np.int32)
ICM_length.shape

(27968, 1)

In [17]:
stacked_URM_all = sps.vstack([stacked_URM_type, ICM_length.T])
stacked_URM_all = sps.csr_matrix(stacked_URM_all)
stacked_URM_all

<41638x27968 sparse matrix of type '<class 'numpy.float64'>'
	with 1600822 stored elements in Compressed Sparse Row format>

In [18]:
stacked_URM_train, stacked_URM_validation = split_train_in_two_percentage_global_sample(stacked_URM_all, train_percentage = 0.8)

In [19]:
recStacked = EASE_R_Recommender(stacked_URM_train)

EASE_R_Recommender: URM Detected 3 ( 0.0%) users with no interactions.
EASE_R_Recommender: URM Detected 130 ( 0.5%) items with no interactions.


In [20]:
recStacked.fit()

EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 16.11 min


In [21]:
evaluator_validation = EvaluatorHoldout(stacked_URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 350 ( 0.8%) Users that have less than 1 test interactions


In [22]:
result_df, _ = evaluator_validation.evaluateRecommender(recStacked)
result_df

EvaluatorHoldout: Processed 41288 (100.0%) in 57.37 sec. Users per second: 720


/kaggle/working/Evaluation/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.065254,0.089647,0.078055,0.032785,0.044367,0.204479,0.092511,0.071083,0.403144,0.256247,...,0.991594,0.399755,0.991594,0.007985,7.83433,0.989068,0.017313,0.577467,5.203801,0.237901
